In [12]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from Functions_George import *

paths = [r"C:\Users\George\Desktop\Final_Project\Images\Control", r"C:\Users\George\Desktop\Final_Project\Images\Penetramax"]

In [13]:
Images = []
for path in paths:
    Images.extend(load_images_from_folder(path))
Images = np.array(Images)
print(Images.shape)

(20, 1200, 1200, 3)


In [14]:
def build_encoder(input_shape, latent_dim):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((3, 3), stride = (3, 3), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((3, 3), stride = (3, 3), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((3, 3), stride = (3, 3), padding='same')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)
    
    return tf.keras.Model(inputs, [z_mean, z_log_var], name='encoder')

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def build_decoder(latent_dim, original_shape):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(150*150*128, activation='relu')(latent_inputs)
    x = layers.Reshape((150, 150, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((3, 3), stride = (3, 3))(x)
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((3, 3), stride = (3, 3))(x)
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((3, 3), stride = (3, 3))(x)
    outputs = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)
    
    return tf.keras.Model(latent_inputs, outputs, name='decoder')

class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var = self.encoder(inputs)
        z = Sampling()([z_mean, z_log_var])
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed
# def build_encoder(input_shape, latent_dim):
#     inputs = layers.Input(shape=input_shape)
#     x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
#     x = layers.MaxPooling2D((2, 2), padding='same')(x)
#     x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#     x = layers.MaxPooling2D((2, 2), padding='same')(x)
#     x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
#     x = layers.MaxPooling2D((2, 2), padding='same')(x)
#     x = layers.Flatten()(x)
#     x = layers.Dense(128, activation='relu')(x)
    
#     z_mean = layers.Dense(latent_dim)(x)
#     z_log_var = layers.Dense(latent_dim)(x)
    
#     return tf.keras.Model(inputs, [z_mean, z_log_var], name='encoder')

# class Sampling(layers.Layer):
#     def call(self, inputs):
#         z_mean, z_log_var = inputs
#         batch = tf.shape(z_mean)[0]
#         dim = tf.shape(z_mean)[1]
#         epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
#         return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# def build_decoder(latent_dim, original_shape):
#     latent_inputs = layers.Input(shape=(latent_dim,))
#     x = layers.Dense(original_shape[0]//8 * original_shape[1]//8 * 64, activation='relu')(latent_inputs)
#     x = layers.Reshape((original_shape[0]//8, original_shape[1]//8, 64))(x)
#     x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     x = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     outputs = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)
    
#     return tf.keras.Model(latent_inputs, outputs, name='decoder')

# class VAE(tf.keras.Model):
#     def __init__(self, encoder, decoder, **kwargs):
#         super(VAE, self).__init__(**kwargs)
#         self.encoder = encoder
#         self.decoder = decoder

#     def call(self, inputs):
#         z_mean, z_log_var = self.encoder(inputs)
#         z = Sampling()([z_mean, z_log_var])
#         reconstructed = self.decoder(z)
#         kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
#         self.add_loss(kl_loss)
#         return reconstructed

input_shape = (1200, 1200, 3)
latent_dim = 128

encoder = build_encoder(input_shape, latent_dim)
decoder = build_decoder(latent_dim, input_shape)
vae = VAE(encoder, decoder)

vae.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

# Assuming `images` is your dataset with shape (num_samples, 1200, 1200, 3)
# Normalize the images
images = Images.astype('float32') / 255.0

vae.fit(images, images, epochs=10 ,batch_size=4)

def generate_images(vae, num_images):
    random_latent_vectors = np.random.normal(size=(num_images, latent_dim))
    generated_images = vae.decoder(random_latent_vectors)
    return generated_images

num_images = 5
new_images = generate_images(vae, num_images)

# Visualize the generated images
for i in range(num_images):
    plt.imshow(new_images[i])
    plt.axis('off')
    plt.show()

Epoch 1/10


In [ ]:
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt
# from tensorflow.keras.applications import VGG16
# from tensorflow.keras.models import Model
# from tensorflow.keras import layers

# # Load pretrained model (e.g., VGG16) without top layers
# base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(1200, 1200, 3))
# base_model.trainable = False  # Freeze the pretrained layers

# def build_encoder(base_model, latent_dim):
#     inputs = base_model.input
#     x = base_model.output
#     x = layers.Flatten()(x)
#     x = layers.Dense(128, activation='relu')(x)
    
#     z_mean = layers.Dense(latent_dim)(x)
#     z_log_var = layers.Dense(latent_dim)(x)
    
#     return tf.keras.Model(inputs, [z_mean, z_log_var], name='encoder')

# class Sampling(layers.Layer):
#     def call(self, inputs):
#         z_mean, z_log_var = inputs
#         batch = tf.shape(z_mean)[0]
#         dim = tf.shape(z_mean)[1]
#         epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
#         return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# def build_decoder(latent_dim, original_shape):
#     latent_inputs = layers.Input(shape=(latent_dim,))
#     x = layers.Dense(original_shape[0]//8 * original_shape[1]//8 * 64, activation='relu')(latent_inputs)
#     x = layers.Reshape((original_shape[0]//8, original_shape[1]//8, 64))(x)
#     x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     x = layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     outputs = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)
    
#     return tf.keras.Model(latent_inputs, outputs, name='decoder')

# class VAE(tf.keras.Model):
#     def __init__(self, encoder, decoder, **kwargs):
#         super(VAE, self).__init__(**kwargs)
#         self.encoder = encoder
#         self.decoder = decoder

#     def call(self, inputs):
#         z_mean, z_log_var = self.encoder(inputs)
#         z = Sampling()([z_mean, z_log_var])
#         reconstructed = self.decoder(z)
#         kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
#         self.add_loss(kl_loss)
#         return reconstructed

# latent_dim = 128

# # Build encoder using pretrained VGG16 model
# encoder = build_encoder(base_model, latent_dim)
# decoder = build_decoder(latent_dim, (1200, 1200, 3))
# vae = VAE(encoder, decoder)

# vae.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

# # Assuming `images` is your dataset with shape (num_samples, 1200, 1200, 3)
# # Normalize the images
# images = Images.astype('float32') / 255.0

# vae.fit(images, images, epochs=10, batch_size=4)

# def generate_images(vae, num_images):
#     random_latent_vectors = np.random.normal(size=(num_images, latent_dim))
#     generated_images = vae.decoder(random_latent_vectors)
#     return generated_images

# num_images = 5
# new_images = generate_images(vae, num_images)

# # Visualize the generated images
# for i in range(num_images):
#     plt.imshow(new_images[i])
#     plt.axis('off')
#     plt.show()


ImportError: Loading a H5 file requires `h5py` to be installed.